Добавляем нули в ID белков для фильтрации

In [53]:
def update_second_column(input_file, output_file):
    updated_lines = []
    with open(input_file, 'r') as f:
        for line in f:
            if line.strip():  # проверяем, что строка не пустая
                parts = line.split()
                if len(parts) >= 2:
                    second_column_value = parts[2]
                    if '_' in second_column_value:
                        prefix, suffix = second_column_value.rsplit('_', 1)
                        if len(suffix) < 5:
                            suffix = suffix.zfill(5)
                        updated_second_column_value = prefix + '_' + suffix
                        parts[2] = updated_second_column_value
                updated_line = ' '.join(parts) + '\n'
                updated_lines.append(updated_line)

    with open(output_file, 'w') as f:
        f.writelines(updated_lines)

# Пример использования:
input_file = '2024_05_07_full_annotation.tsv'
output_file = 'output_all.txt'
update_second_column(input_file, output_file)

Восстановление регионов брекс

In [62]:
def get_number_from_protein(protein_string):
    # Находим второе появление символа "_" и извлекаем число после него
    second_underscore_index = protein_string.find('_', protein_string.find('_') + 1)
    number_string = protein_string[second_underscore_index + 1:]
    # Преобразуем строку в число, учитывая ведущие нули
    return int(number_string.lstrip('0'))

def write_to_output_file(clust_region_list, output_file):
    with open(output_file, 'a') as f:
        f.write(','.join(clust_region_list) + '\n')

def main(input_file, output_file):
    # Сортируем файл по второй колонке
    with open(input_file, 'r') as f:
        lines = f.readlines()
        # Пропускаем первую строку (хэдер)
        header = lines[0]
        data = sorted(lines[1:], key=lambda x: x.split(' ')[2])

    clust_region = []
    prev_protein_number = None

    for line in data:
        # Разделяем строку по пробелу
        columns = line.strip().split(' ')
        # Проверяем, есть ли четыре столбца
        if len(columns) < 3:
            print("Ошибка: Неверное количество столбцов в строке:", line)
            continue
        protein = columns[2]
        size_clust = int(columns[1])
        clust_region_value = columns[0]
        if size_clust <= 0:
            clust_region_value = 'x' + clust_region_value

        # Получаем число из второй колонки (protein)
        try:
            protein_number = get_number_from_protein(protein)
        except ValueError:
            print("Ошибка: Невозможно извлечь число из строки:", protein)
            continue

        if prev_protein_number is None:
            prev_protein_number = protein_number

        # Проверяем разницу между текущим и предыдущим числами
        if abs(protein_number - prev_protein_number) <= 1:
            # Добавляем значение в лист clust_region
            clust_region.append(clust_region_value + '_' + columns[-2])  # Взять значение из последней колонки
        else:
            # Разница больше 1, записываем текущий список в новый файл
            write_to_output_file(clust_region, output_file)
            # Очищаем список
            clust_region = []
            # Добавляем текущее значение в новый список
            clust_region.append(clust_region_value + '_' + columns[-2])  # Взять значение из последней колонки

        prev_protein_number = protein_number

    # Записываем оставшиеся данные в файл
    write_to_output_file(clust_region, output_file)

if __name__ == '__main__':
    input_file = 'output_all.txt'
    output_file = 'output_file_full_all.csv'
    main(input_file, output_file)

Обработка последовательностей кластеров

In [63]:
import csv

def reverse_proteins(line):
    proteins = line.strip().split(',')
    inner_index = None
    brex_index = None
    for i, protein in enumerate(proteins):
        if protein.endswith('_inner') or protein.endswith('_brex'):
            inner_index = i
            brex_index = i
            break
    if inner_index is not None:
        downstream_index = None
        for i, protein in enumerate(proteins):
            if protein.endswith('_downstream'):
                downstream_index = i
                break
        if downstream_index is not None and downstream_index < inner_index:
            reversed_proteins = proteins[::-1]
            return ','.join(reversed_proteins)
    return line.strip()

input_file = 'output_file_full_all.csv'
output_file = 'output_file_full_ordered_all.csv'

with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
    reader = csv.reader(f_in)
    writer = csv.writer(f_out)
    for line in reader:
        reversed_line = reverse_proteins(','.join(line))
        writer.writerow(reversed_line.split(','))


запись только внутренних

In [64]:
def filter_csv(input_file, output_file):
    filtered_rows = []

    with open(input_file, 'r', newline='') as f_in:
        reader = csv.reader(f_in)
        for row in reader:
            filtered_row = [item for item in row if item.endswith('_brex') or item.endswith('_inner')]
            filtered_rows.append(filtered_row)

    with open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerows(filtered_rows)

# Пример использования:
input_file = 'output_file_full_ordered_all.csv'
output_file = 'output_full_filtered_all.csv'
filter_csv(input_file, output_file)

Замена имен кластеров на "singletone"

In [65]:
def process_csv(input_file, output_file):
    with open(input_file, 'r') as infile:
        with open(output_file, 'w') as outfile:
            for line in infile:
                elements = line.strip().split(',')
                modified_elements = [element if not element.startswith('x') else 'SINGLETONE' for element in elements]
                modified_line = ','.join(modified_elements) + '\n'
                outfile.write(modified_line)

# Использование функции для обработки CSV файла
process_csv('output_full_filtered_all.csv', 'full_singletone_all.csv')

Подсчет уникальных регионов и запись их в файл

In [66]:
import csv
from collections import Counter

def process_protein_names(names):
    # Удаляем приписки "_inner" и "_brex" и сортируем имена белков
    processed_names = [name.replace("_inner", "").replace("_brex", "") for name in names]
    return tuple(processed_names)

def count_unique_regions(file_path):
    regions_counter = Counter()

    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            # Преобразуем имена белков и считаем уникальные регионы
            region = process_protein_names(row)
            regions_counter[region] += 1

    # Сортируем словарь по значениям (количество встречаемости региона) в убывающем порядке
    sorted_regions_counter = dict(sorted(regions_counter.items(), key=lambda item: item[1], reverse=True))

    # Выводим результаты
    print("Unique regions count:")
    for region, count in sorted_regions_counter.items():
        print(f"{region}: {count}")
    return sorted_regions_counter

file_path = "full_singletone_all.csv"  # Замените путь на путь к вашему файлу CSV
regions_counter = count_unique_regions(file_path)

Unique regions count:
('CLUS_NZ_CP004887.1_50', 'CLUS_NZ_CP024960.1_713', 'CLUS_NZ_LR134303.1_3991', 'CLUS_NC_014376.1_3647', 'CLUS_NZ_CP020472.1_3338', 'CLUS_NZ_CP048433.1_824'): 213
('CLUS_NZ_CP004887.1_50', 'CLUS_NZ_CP024960.1_713', 'CLUS_NZ_LR134303.1_3991', 'CLUS_NC_014376.1_3647', 'CLUS_NZ_CP084664.1_3537', 'CLUS_NZ_AP025784.1_505', 'CLUS_NZ_CP020472.1_3338', 'CLUS_NZ_CP048433.1_824'): 133
('CLUS_NZ_CP019457.1_5609', 'CLUS_NC_017030.1_5257', 'CLUS_NZ_CP050693.1_2806', 'CLUS_NZ_CP064873.1_2768', 'CLUS_NZ_CP031142.1_4094', 'CLUS_NZ_CP024989.1_6459'): 42
('CLUS_NZ_CP004887.1_50', 'CLUS_NZ_CP024960.1_713', 'CLUS_NZ_LR134303.1_3991', 'CLUS_NZ_LS483250.1_1673', 'CLUS_NC_014376.1_3647', 'CLUS_NZ_CP020472.1_3338', 'CLUS_NZ_CP048433.1_824'): 42
('CLUS_NZ_LR215983.1_4173', 'CLUS_NZ_CP024960.1_713', 'CLUS_NZ_LR134303.1_3991', 'CLUS_NC_014376.1_3647', 'CLUS_NC_008346.1_2567', 'CLUS_NZ_CP048433.1_824'): 38
('CLUS_NZ_CP004887.1_50', 'CLUS_NZ_CP024960.1_713', 'CLUS_NZ_LR134303.1_3991', 'CLUS_NC

In [68]:
def write_regions_to_file(regions_counter, output_file_path):
    with open(output_file_path, 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)
        for region, count in regions_counter.items():
            # Записываем одну строку для каждого региона
            row = list(region) + [count]
            csv_writer.writerow(row)
output_file_path = "full_singletone_count_all.csv"
write_regions_to_file(regions_counter, output_file_path)

Запись уникальных кластеров по аннотации

In [69]:
def read_txt_file(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        # Пропускаем заголовок
        next(file)
        for line in file:
            # Разбиваем строку по пробелам
            parts = line.strip().split()
            # Получаем ключ (первая колонка) и значение (пятая колонка)
            key = parts[0]
            value = parts[3]
            # Добавляем в словарь
            data_dict[key] = value
        data_dict['SINGLETONE'] = 'singletone'
    return data_dict

def process_file(input_file_path, output_file_path, data_dict):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        for line in input_file:
            keys = line.strip().split(',')
            # Игнорируем последнее число в строке
            count = keys[-1]
            keys = keys[:-1]
            result_line = []
            for key in keys:
                if key in data_dict:
                    result_line.append(data_dict[key])
            # Записываем строку в выходной файл с добавлением последнего числа из словаря
            output_file.write(','.join(result_line) + ',' + count + '\n')

# Пример использования:
file_path = 'output_all.txt'  # Замените на путь к вашему файлу
data_dict = read_txt_file(file_path)

input_file_path = 'full_singletone_count_all.csv'  # Замените на путь к вашему входному файлу
output_file_path = 'full_single_annot_reg_count_all.txt'  # Замените на путь к вашему выходному файлу
process_file(input_file_path, output_file_path, data_dict)